<a href="https://colab.research.google.com/github/rogerlga/mvp-adbp/blob/main/mvp_adbp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP Análise de Dados e Boas Práticas

**Nome:** Roger Luis Giroldo Assunção

**Matrícula:** 4052025000584

**Dataset:** Dados operacionais de Turbogerador - Adendo A.3 da oportunidade nº 7004066211 do [Portal Petronect](https://www.petronect.com.br/irj/go/km/docs/pccshrcontent/Site%20Content%20(Legacy)/Portal2018/pt/lista_licitacoes_concluidas.html)

# **Descrição do problema**

## Dataset

Iremos trabalhar com dados de uma operação industrial. Faremos uso de dados disponibilizados publicamente em um processo de licitação da Petrobras, acessível em:

> [Portal Petronect](https://www.petronect.com.br/irj/go/km/docs/pccshrcontent/Site%20Content%20(Legacy)/Portal2018/pt/lista_licitacoes_concluidas.html)<br>
> Nº da oportunidade = 7004066211

Dentre os datasets disponibilizados, analisaremos o dataset do "Adendo A.3", referente a um trem de equipamentos para geração de energia: um turbogerador composto por turbina a gás, caixa de engrenagem e gerador elétrico.

## Descrição

O esquema abaixo representa o trem de equipamentos objeto da análise, formado por:

- Turbina a Gás - composta pelo GG (Gas Generator) e a PT (Power Turbine),
- Caixa de Engrenagem (Gearbox), e
- Gerador Elétrico (Generator).

<img src="https://raw.githubusercontent.com/rogerlga/mvp-adbp/refs/heads/main/assets/two_shaft_turbogenerator.jpg" width="800">

*Figura 1 - Trem de equipamentos (fonte: http://emadrlc.blogspot.com/2013/01/chapter-1-introduction-to-gas-turbines.html)*

As variáveis ou atributos do dataset são medições de instrumentos/sensores instalados nesses equipamentos. A essas variáveis nos referimos por "tags".

Uma análise inicial das tags do conjunto completo acima será realizada e na sequência focaremos nas variáveis da Turbina a Gás - GG (Gas Generator) + PT (Power Turbine).

### Turbina a Gás

Os instrumentos da Turbina a Gás são descritos a seguir (as referências numéricas encontram-se na próxima figura):

1. Abertura da válvula de controle, pressões e temperatura do gás combustível;
2. Pressões e temperatura do combustível líquido (diesel);
3. Pressão e temperatura na sucção do compressor;
4. Pressão e posições de VSVs (Variable Stator Vanes) do compressor;
5. Vibração do GG;
6. Pressão e temperatura na descarga do compressor;
7. Pressão e temperatura na exaustão do GG;
8. Temperatura do ar de refrigeração da PT;
9. Vibração da PT;
10. Temperatura na exaustão da PT;
11. Temperatura do óleo no sump A;
12. Pressões e temperaturas na AGB (Accessory GearBox);
13. Temperatura do óleo no sump B;
14. Pressões do ar de refrigeração do GG;
15. Temperatura do óleo no sump C;
16. Temperaturas do 2º estágio da PT;
17. Posição axial da PT;
18. Temperaturas do mancal axial da PT;
19. Temperatura do ar na saída do compartimento da turbina.

A figura a seguir apresenta a localização na Turbina a Gás dos instrumentos do dataset em análise, agrupados conforme numeração acima:

<img src="https://raw.githubusercontent.com/rogerlga/mvp-adbp/refs/heads/main/assets/TG_tags.png" width="900">

*Figura 2 - Representação de uma Turbina a Gás com os atributos do dataset (fonte: adaptado de https://www.researchgate.net/figure/Schematic-of-LM2500-marine-gas-turbine_fig1_349497740 pelo autor)*

## Hipóteses

As hipóteses referentes ao problema são:

- Existe correlação entre as variáveis, de forma que futuramente poderemos predizer um atributo target a partir dos demais;
- Não há dados anômalos suficientes na indústria, o que dificulta treinos supervisionados para detecção de anomalias;
- É possível distinguir diferentes perfis operacionais através dos dados, visando futuramente treinar um modelo que faça a classificação automática desses perfis.

## Objetivos

Os objetivos desse estudo vão permear em torno de preparar o dataset para:

- Treinar um modelo **supervisionado** que preveja os valores de temperatura de exaustão da turbina a partir das demais tags (em condição normal), visando comparar valores reais com as predições, o que dará um indicativo da eficiência do equipamento;
- Treinar um modelo **não supervisionado** que detecte os períodos em que o equipamento está apresentando comportamento anômalo, pré-falha. Deverá ser não supervisionado, com base no período sabido como "normal", pois não há rótulos de anomalias, e será considerado anômalos os comportamentos diferentes daquele aprendido como "normal".
- Treinar um modelo **nâo supervisionado** para classificação automática dos distintos perfis operacionais, como operação com gás, operação com diesel, turbina em carga base, em carga parcial, em partida, parada, etc.

# **Pacotes e Carregamento**

## Instalação e importação de pacotes

In [ ]:
!wget --no-cache -P utils/ "https://raw.githubusercontent.com/rogerlga/mvp-adbp/refs/heads/main/utils/plotter.py"

In [35]:
# Pacote do projeto, contendo utilitários
from utils import plotter

# Pacotes instalados
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

### Carregamento dos dados

In [36]:
df = pd.read_csv("https://raw.githubusercontent.com/rogerlga/mvp-adbp/refs/heads/main/Adendo%20A.3_Conjunto%20de%20Dados_DataSet.csv")

# **Análise Exploratória**

## Geral

Primeiramente analisaremos o dataset como um todo, a começar por olhar a disposição geral dos dados.

In [ ]:
df

Percebemos que os dados possuem uma classificação dada pela 1ª coluna "role", que a 2ª coluna "offset_seconds" é uma escala temporal e que as outras 79 colunas são as "tags" (terminologia da indústria), isto é, variáveis referentes às medições dos sensores/instrumentos do turbogerador. Cada linha do dataset representa um instante de tempo, contendo a medição de cada sensor correspondente a esse instante.

Seguiremos com todas as variáveis nessa primeira fase de análise e no momento oportuno focaremos nas variáveis da Turbina a Gás apenas, conforme as tags que foram listadas e representadas na descrição do início desse trabalho.

### Tipos

In [ ]:
df.dtypes

Os tipos inferidos automaticamente parecem estar corretos, considerando que a coluna "role" é uma classificação (tipo objeto/texto), a coluna "offset_seconds" é uma escala temporal em segundos (tipo inteiro) e as outras colunas são valores medidos pelos instrumentos, podendo ser números reais ou inteiros.

### Informações básicas

Em seguida, algumas informações e estatísticas descritivas serão levantadas para nos dar uma noção inicial da composição dos dados. 

In [ ]:
df.info()

No total temos 23311 instâncias, sendo que nenhuma variável possui valor nulo. No entanto, no caso das tags, valores discrepantes muito negativos ou muito positivos ou o valor zero podem representar problemas de leitura dos sensores e serem interpretados como nulos/inválidos. Faremos essa avaliação no decorrer da análise abaixo.

In [ ]:
df.describe()

De início, notamos que as tags possuem escalas bem diferentes. Por exemplo, o instrumento PIT-591D varia entre -0.00175 e 0.00486 apenas, enquanto que o TE-508D varia de 28.073 a 833.696.

Outros dois pontos que chamam a atenção são o fato do sensor ZT-502D-B conter apenas valores 0.0 e o ZT-503D-B conter um mínimo de -999.0, valor muito discrepante quando consideramos a média e o desvio padrão da coluna (24.617 e 15.279, respectivamente).

Graficamente, podemos ter a visualização das estatísticas acima através do gráfico boxplot:

In [ ]:
# Define área de plotagem e o gráfico boxplot
plt.figure(figsize=(18, 12))
sns.boxplot(df.drop(columns=['role', 'offset_seconds']), orient="v", fliersize=2)

# Deixa os rótulos no eixo x no sentido vertical
plt.xticks(rotation=90)

plt.show()

As mesmas observações anteriores, como a diferença nas escalas e o outlier da tag ZT-503DB, ficam ainda mais evidentes na visualização gráfica acima.

## Discriminação por papel ("role")

Os dados originais possuem uma divisão entre diferentes papéis ou propósitos, dada pela coluna "role".

Veremos primeiramente quais são esses papéis e a quantidade de instâncias para cada um deles:

In [ ]:
df.groupby("role")["role"].count()

Podemos obter também uma estatística descritiva separada para cada papel:

In [ ]:
df.groupby("role").describe().stack(future_stack=True)

Porém, uma forma mais efetiva de avaliar as estatísticas acima é através de boxplots de cada tag, separadas por papel.

Primeiramente, vamos criar uma versão do dataframe no formato "long":

In [ ]:
# Transforma o formato do dataframe de "wide" para "long" e armazena o resultado em uma nova variável
df_long = df.melt(id_vars=["role", "offset_seconds"], var_name="tag", value_name="value")

# Cria uma coluna "instrument" que classifica as instâncias pelo tipo do sensor/instrumento
# Obs.: é extraído o prefixo das tags, que denota o tipo do instrumento, por exemplo: PT para transmissor de pressão, TE para sensor de temperatura, etc.
df_long["instrument"] = df_long["tag"].str.split('-', expand=True)[0]

df_long

O dataframe no formato "long" facilita a geração facetada dos gráficos boxplot, assim como a identificação por cores e outros elementos estéticos e geométricos.

In [ ]:
# Executa a função catplot do seaborn para geração de visualização facetada
g = sns.catplot(df_long, x="value", y="role", hue="instrument", col="tag", col_wrap=5, kind="box", sharex=False, height=1.5, aspect=2, fliersize=2)

# Reposiciona a legenda
sns.move_legend(g, "lower center", bbox_to_anchor=(.5, 1), ncol=20)

plt.show()

Aqui repetimos o que já foi percebido anteriormente e acrescentamos mais algumas observações:

- ZT-502D-B possui apenas valores iguais a zero;
- ZT-503D-B possui valor inválido (outlier) próximo de -1000.0 no grupo "test-1";
- PIT-562D possui potenciais outliers no grupo "test-1" entre os valores 0.0 e 4.0;
- XT-508D-X possui potencial outlier no grupo "test-1" com valor superior a 25.0;
- Grupos "normal" e "test-0" são iguais, visto que, além de possuírem a mesma quantidade de instâncias (observado anteriormente), possuem gráficos boxplot idênticos para todas as tags.

Para cravar quanto à conclusão do último bullet acima, precisamos abranger a variável temporal "offset_seconds" na análise. Para tal, vamos plotar a tendência de cada variável e verificar se também são idênticas entre os grupos "normal" e "test-0":


In [ ]:
# Executa a função relplot do seaborn para geração de visualização facetada
g = sns.relplot(df_long[df_long["role"] != "test-1"], x="offset_seconds", y="value", hue="role", style="role", col="tag", col_wrap=5, kind="line", height=1.5, aspect=2, facet_kws=dict(sharey=False))

# Reposiciona a legenda
sns.move_legend(g, "lower center", bbox_to_anchor=(.5, 1), ncol=20)

plt.show()

As linhas dos grupos "normal" e "test-0" ficaram perfeitamente sobrepostas para todas as tags. Pontanto, confirmamos a conclusão de que esses dois grupos contêm dados idênticos.

## Matriz de correlação

Calcularemos a correlação linear (correlação de Pearson) entre todas as variáveis:

In [ ]:
corr = df[df["role"] != "test0"].drop(columns=["role", "offset_seconds"]).corr()
corr

In [ ]:
plt.figure(figsize=(20, 18))
sns.heatmap(corr, annot=False, cmap='coolwarm', fmt=".1f")
plt.show()

Notamos que há uma forte correlação entre diversos núcleos de variáveis, o que nos mostra que há grande potencial de reduzir a quantidade de atributos através do descarte de variáveis redundantes ou similares ou através de feature engineering, como agregações entre variáveis.

# **Pré-processamento**

## Tratamento geral e de nulos/inválidos

Primeiramente, descartaremos as instâncias com "role" igual à "test-0", pois são idênticas às instâncias em que "role" é igual à "normal", conforme vimos na análise exploratória:

In [ ]:
df2 = df[df["role"] != "test-0"].copy()
df2["role"].unique()

Esse é o momento também em que focaremos apenas nas variáveis da Turbina a Gás, visto que o foco do estudo e das hipóteses é em cima deste equipamento. Futuramente, quando iniciar o trabalho de modelagem e a depender dos resultados dos modelos, podemos voltar à fase de análise exploratória e pré-processamento de forma iterativa, considerando a possibilidade de capturar alguma variável dos equipamentos adjacentes que possa ajudar a melhorar o modelo.

In [ ]:
df2 = df2[[
    "role", "offset_seconds",
    "FCV-501D_ACT", "PIT-501D", "PIT-516D", "PIT-517D", "PIT-518D", "PIT-519D", "PIT-521D", "PIT-523D",
    "PIT-529D", "PIT-530D", "PIT-532D", "PT-502D", "PT-503D", "PT-525D", "TE-504D", "TE-506D",
    "TE-507D-A", "TE-508D", "TE-509D", "TE-514D-A", "TE-515D-A", "TE-516D", "TE-517D", "TE-521D",
    "TE-522D", "TE-523D", "TE-524D", "TE-525D", "TE-526D", "TIT-501D", "TIT-505D", "TIT-561D",
    "VT-501D", "VT-502D", "ZT-502D-B", "ZT-503D-A", "ZT-503D-B", "ZY-503D"
]]

Como vimos anteriormente, os dados não possuem valores nulos, mas a tag ZT-502D-B, que pertence à Turbina a Gás, contém apenas zeros. Dessa forma, iremos eliminá-la:

In [ ]:
df2.drop(columns="ZT-502D-B", inplace=True)

Por fim, atualizaremos a versão "long" do dataframe:

In [ ]:
# Transforma o formato do dataframe de "wide" para "long" e armazena o resultado em uma nova variável
df2_long = df2.melt(id_vars=["role", "offset_seconds"], var_name="tag", value_name="value")
df2_long["role"].unique()

## Sensores da Turbina a Gás

Para alguns conjuntos de sensores da Turbina a Gás, iremos avaliar os dados e deliberar se podemos descartar algum atributo ou gerar novos atributos (feature engineering) a partir dos existentes. Para uma melhor avaliação, foi utilizado o pacote *Plotly* na geração dos gráficos, permitindo a interação e facilitando a análise dos dados de série temporal.

Os grupos estão definidos conforme Figura 2 mostrada na parte introdutória desse trabalho.

### Grupos 1 e 2

In [ ]:
# Filtra o dataframe e cria atributos auxiliares para o plot
plot_data = plotter.prepare_timeseries_data(df2_long, lambda x: {
    "gas_valve":    (x["tag"] == "FCV-501D_ACT"), # Vávlula gás (grupo 1)
    "gas_press":    (x["tag"] == "PIT-501D") | (x["tag"] == "PT-502D") | (x["tag"] == "PT-503D"), # Pressões gás (grupo 1)
    "gas_temp":     (x["tag"] == "TIT-501D"), # Temperatura gás (grupo 1)
    "diesel_press": (x["tag"] == "PIT-516D") | (x["tag"] == "PIT-517D"), # Pressões diesel (grupo 2)
    "diesel_temp":  (x["tag"] == "TE-504D"), # Temperatura diesel (grupo 2)
})

# Plota o gráfico
fig = plotter.timeseries(plot_data, "time", "value", pd.to_timedelta("600s"),
                         color="tag", facet_row="class", facet_col="role",
                         sync_xaxes="by_col", sync_yaxes="by_row", height=800)
fig.show()

Os subplots foram separados por "role" e por um atributo auxiliar criado ("class"), que nesse plot específico classifica pelo combustível e tipo de instrumento aos quais as tags estão relacionadas.

A tag FCV-501D_ACT é a variável que representa o controle de injeção de gás. Ela tem a capacidade de definir perfis operacionais, além de poder ser o atributo target em alguns modelos. Portanto, será mantida no dataset.

Dentre as tags PIT-501D, PT-502D e PT-503D (pressões de gás), que são bastante relacionadas, manteremos apenas a PT-503D, visto que é a pressão percebida pelo combustor da turbina e influencia mais diretamente na condição do equipamento. Além disso, a tag PIT-501D por vezes parece apresentar dados não condizentes com o status operacional da turbina, quando a válvula de gás está fechada (FCV-501D_ACT ~= 0.0), mas o instrumento continua indicando pressão, o que provavelmente denota uma medição do gás no ponto de chegada/fornecimento, que pode ficar isolado do sistema da Turbina a Gás em determinadas condições.

Com relação às tags PIT-516D e PIT-517D, apesar de serem relacionadas, ambas serão mantidas, visto que medem pressões de diesel em manifolds diferentes e aparentam possuir valores coerentes.

Há apenas um atributo de medição de temperaturas para cada combustível (tags TIT-501D e TE-504D), por isso elas também permanecem, considerando que os valores aparentam coerentes.

A execução dos tratamentos descritos acima são feitos na sequência:

In [ ]:
df2.drop(columns=["PIT-501D", "PT-502D"], inplace=True)

### Grupo 4

Conforme vimos na análise exploratória, a tag ZT-503D-B, que pertence a esse grupo, possui um outlier (valor = -999.0). Iremos removê-lo do dataframe de formato "long" antes de gerar os gráficos de série temporal, para que ele não afete as escalas e dificulte a análise.

In [ ]:
outlier_condition = (df2_long["tag"] == "ZT-503D-B") & (df2_long["value"] < -900.0)
df2_long = df2_long[~outlier_condition]

In [ ]:
# Filtra o dataframe e cria atributos auxiliares para o plot
plot_data = plotter.prepare_timeseries_data(df2_long, lambda x: {
    "VSV_press": (x["tag"] == "PIT-532D"), # Pressão atuador VSV
    "VSV_pos":   (x["tag"] == "ZT-503D-A") | (x["tag"] == "ZT-503D-B") | (x["tag"] == "ZY-503D"), # Posições VSV
})

# Plota o gráfico
fig = plotter.timeseries(plot_data, "time", "value", pd.to_timedelta("600s"),
                         color="tag", facet_row="class", facet_col="role",
                         sync_xaxes="by_col", sync_yaxes="by_row", height=400)
fig.show()

Manteremos a tag PIT-532D, por ser o único dado de pressão do sistema de VSV e por apresentar valores coerentes.

As tags de posição das VSVs (ZT-503D-A, ZT-503D-B e ZY-503D) possuem curvas praticamente iguais, o que indica uma redundância. Iremos manter apenas a variável que possui valores mais centrais dentre as três tags: a ZY-503D. Ao eliminar as outras duas, também não precisamos nos preocupar em remover o outlier mencionado acima do dataframe de formato "wide", já que a tag que contém ele será eliminada.

Segue a execução dos tratamentos descritos acima:

In [ ]:
df2.drop(columns=["ZT-503D-A", "ZT-503D-B"], inplace=True)

### Grupos 8, 14 e 16

In [ ]:
# Filtra o dataframe e cria atributos auxiliares para o plot
plot_data = plotter.prepare_timeseries_data(df2_long, lambda x: {
    "HPT_cool_press": (x["tag"] == "PIT-518D") | (x["tag"] == "PIT-519D"), # Pressões refrigeração HPT (grupo 14)
    "PT_cool_temp":   (x["tag"] == "TIT-505D"), # Temperatura refrigeração PT (grupo 8)
    "PT_temp":        (x["tag"] == "TE-516D") | (x["tag"] == "TE-517D"), # Temperaturas PT (grupo 16)
})

# Plota o gráfico
fig = plotter.timeseries(plot_data, "time", "value", pd.to_timedelta("600s"),
                         color="tag", facet_row="class", facet_col="role",
                         sync_xaxes="by_col", sync_yaxes="by_row", height=540)
fig.show()

Pelas características observadas, aqui faremos a média das pressões de refrigeração (PIT-518D e PIT-519D) e manteremos a tag de temperatura de refrigeração da PT (TIT-505D), visto ser a única para este dado.

Também manteremos as duas variáveis de temperatura da roda de 2º estágio da PT (TE-516D e TE-517D) no dataset, visto que são medições em duas posições diferentes internamente à turbina e apresentam comportamentos distintos em determinadas situações.

Os referidos tratamentos seguem abaixo:

In [ ]:
df2["PIT-518D_519D"] = df2[["PIT-518D", "PIT-519D"]].mean(axis=1)
df2.drop(columns=["PIT-518D", "PIT-519D"], inplace=True)

### Grupo 12

In [ ]:
# Filtra o dataframe e cria atributos auxiliares para o plot
plot_data = plotter.prepare_timeseries_data(df2_long, lambda x: {
    "AGB_press": (x["tag"] == "PIT-529D") | (x["tag"] == "PIT-530D"), # Pressões óleo AGB
    "AGB_temp":  (x["tag"] == "TE-521D") | (x["tag"] == "TE-522D") | (x["tag"] == "TE-526D"), # Temperaturas óleo AGB
})

# Plota o gráfico
fig = plotter.timeseries(plot_data, "time", "value", pd.to_timedelta("600s"),
                         color="tag", facet_row="class", facet_col="role",
                         sync_xaxes="by_col", sync_yaxes="by_row", height=400)
fig.show()

Apesar de nem todas as variáveis desse grupo serem redundantes, elas possuem comportamento muito semelhante. Como elas acompanham a mesma tendência, mas os patamares de valores são ligeiramente diferentes, lançaremos mão de uma feature engineering básica.

Vamos adotar a média das pressões (tags PIT-529D e PIT-530D), o máximo entre as temperaturas do óleo da AGB (TE-521D e TE-522D) e manteremos a variável de temperatura do cárter (TE-526D) sem modificação.

Seguem os tratamentos mencionados:

In [ ]:
df2["PIT-529D_530D"] = df2[["PIT-529D", "PIT-530D"]].mean(axis=1)
df2["TE-521D_522D"] = df2[["TE-521D", "TE-522D"]].max(axis=1)
df2.drop(columns=["PIT-529D", "PIT-530D", "TE-521D", "TE-522D"], inplace=True)

### Demais grupos

In [ ]:
import importlib
importlib.reload(plotter)

# Filtra o dataframe e cria atributos auxiliares para o plot
plot_data = plotter.prepare_timeseries_data(df2_long, lambda x: {
    "Eff_press": (x["tag"] == "PIT-521D") | (x["tag"] == "PT-525D") | (x["tag"] == "PIT-523D"), # Pressões eficiência GG (grupos 3, 6 e 7)
    "Eff_temp":  (x["tag"] == "TE-506D") | (x["tag"] == "TE-507D-A") | (x["tag"] == "TE-508D") | (x["tag"] == "TE-509D"), # Temperaturas eficiência GG e PT (grupos 3, 6, 7 e 10)
    "Sump_temp": (x["tag"] == "TE-523D") | (x["tag"] == "TE-524D") | (x["tag"] == "TE-525D"), # Temperaturas Sumps GG (grupos 11, 13 e 15)
    "Vib":       (x["tag"] == "VT-501D") | (x["tag"] == "VT-502D"), # Vibrações GG e PT (grupos 5 e 9)
    "Hood_temp": (x["tag"] == "TIT-561D"), # Temperatura compartimento Turbina (grupo 19)
})

# Plota o gráfico
fig = plotter.timeseries(plot_data, "time", "value", pd.to_timedelta("600s"),
                         color="tag", facet_row="class", facet_col="role",
                         sync_xaxes="by_col", sync_yaxes="by_row", height=800)
fig.show()

Com relação às demais tags, divididas logicamente acima, percebemos que os dados de todas elas parecem coerentes e nenhum tratamento em relação a isso precisa ser realizado.

Avaliando quanto ao possível descarte ou feature engineering, não convém realizar nenhum também, visto que nenhuma é medição redundante e, apesar de vermos alguns comportamentos similares, apresentam respostas distintas a determinadas situações, o que é uma característica importante para uma futura modelagem.

## Split dos dados

Dado que uma das modelagens pretendidas a se fazer com este dataset é uma análise de regressão, para predição das temperaturas de exaustão da turbina (exaustão da GG e exaustão da PT) a partir das demais variáveis, torna-se necessária a divisão entre features (X) e targets (y).

Para este caso, os dados usados serão somente os considerados "normais", visto que queremos usar essa predição para ter uma indicação da eficiência do equipamento, comparando os valores de temperatura reais com os valores previstos pelo modelo, que deve aprender a prever as temperaturas nos casos de operação saudável apenas.

In [ ]:
X = df2[df2["role"] == "normal"].drop(columns=["role", "offset_seconds", "TE-508D", "TE-509D"])
y1 = df2.loc[df2["role"] == "normal", "TE-508D"]
y2 = df2.loc[df2["role"] == "normal", "TE-509D"]

In [ ]:
X_train, X_test, y1_train, y1_test, y2_train, y2_test = train_test_split(X, y1, y2, test_size=0.2, random_state=42)

print(f"Dimensões de X_train: {X_train.shape}")
print(f"Dimensões de X_test: {X_test.shape}")
print(f"Dimensões de y1_train: {y1_train.shape}")
print(f"Dimensões de y1_test: {y1_test.shape}")
print(f"Dimensões de y2_train: {y2_train.shape}")
print(f"Dimensões de y2_test: {y2_test.shape}")

## Normalização

Considerando que os atributos possuem escalas bem diferentes entre si e que alguns algoritmos de Machine Learning são sensíveis a isso, normalizaremos os dados:

In [ ]:
scaler_norm = MinMaxScaler()

# Aprende min e max de X_train
scaler_norm.fit(X_train)

# Aplica a normalização
X_train_normalized = scaler_norm.transform(X_train)
X_test_normalized = scaler_norm.transform(X_test)

# Transforma em dataframe e exibe
df_normalized = pd.DataFrame(X_train_normalized, columns=X_train.columns)
df_normalized

In [ ]:
plot_data = pd.concat(
    [X_train[["PT-525D"]], df_normalized[["PT-525D"]].rename(columns={ "PT-525D": "PT-525D_normalized" })], axis=1
).melt(var_name="tag", value_name="value")
g = sns.FacetGrid(plot_data, col="tag", sharex=False, height=5, aspect=1)
g.map(sns.histplot, "value", kde=True)
plt.show()

O comparativo de histogramas acima mostra que os valores da tag PT-525D foram escalados para o intervalo de 0 a 1, mantendo a forma da distribuição original.

## Padronização

De forma similar, considerando que alguns algoritmos de Machine Learning são sensíveis à escala das características, faremos a padronização:

In [ ]:
scaler_std = StandardScaler()

# Aprende min e max de X_train
scaler_std.fit(X_train)

# Aplica a padronização
X_train_standardized = scaler_std.transform(X_train)
X_test_standardized = scaler_std.transform(X_test)

# Transforma em dataframe e exibe
df_standardized = pd.DataFrame(X_train_standardized, columns=X_train.columns)
df_standardized

In [ ]:
plot_data = pd.concat(
    [X_train[["PT-525D"]], df_standardized[["PT-525D"]].rename(columns={ "PT-525D": "PT-525D_standardized" })], axis=1
).melt(var_name="tag", value_name="value")
g = sns.FacetGrid(plot_data, col="tag", sharex=False, height=5, aspect=1)
g.map(sns.histplot, "value", kde=True)
plt.show()

print(df_standardized["PT-525D"].describe(percentiles=[]))

O comparativo de histogramas acima e a estatística descritiva mostram que os valores da tag PT-525D foram transformados para resultar em média próxima de zero e desvio padrão próximo de um, centralizando a distribuição.

# **Conclusão**

O dataset estudado e tratado acima possui dados temporais referentes à operação de um turbogerador. O trabalho fez uma análise inicial do trem de máquinas completo e depois focou no objetivo principal: variáveis da Turbina a Gás.

Os dados foram limpos e tratados, considerando redundâncias/similaredades entre as medições, além do uso de feature engineering, sobretudo para agregar variáveis similares. Os datasets finais - 1 de treino, 1 de teste proveniente do split dos dados "normais" e 1 de teste proveniente do atributo "role", que possui alguma anomalia operacional - contêm 30 variáveis contínuas, referentes a medições de sensores (tags) da Turbina a Gás.

Na análise podemos notar que existe bastante correlação entre as variáveis de interesse, confirmando a primeira hipótese mencionada na descrição do problema. Além disso, a anomalia do equipamento que supostamente há nos dados (mais especificamente nas instâncias "test-1"), não é de fácil identificação através da análise exploratória, nem são os dados rotulados com anomalia, o que reforça a segunda hipótese. Também notamos mudanças de patamares nos valores das variáveis em determinados momentos, o que indica ser possível identificar os perfis operacionais através dos dados, conforme terceira hipótese.

Os datasets estão prontos para serem usados numa modelagem inicial, visando atender aos objetivos traçados no início desse documento, como identificar diferentes perfis operacionais (não supervisionado - agrupamento), identificação de anomalias (não supervisionado, dada a abordagem planejada de usar apenas dados "normais" no treino, diante da escassez de anomalias e da dificuldade em se obter rótulos de anomalia) e regressão para predição das temperaturas de exaustão com base nas demais variáveis (supervisionado).